This is where we will define a quantum multiplier.

Step 1: Set up environment

In [1]:
# !pip install cirq
import cirq
from cirq.circuits import InsertStrategy
from cirq import H, SWAP, CZPowGate
from cirq import X

Step 2: Implementing QFT and IQFT

In [2]:
def iqft(n,qubits,circuit):
    #Swap the qubits
    for i in range(n//2):
        circuit.append(SWAP(qubits[i],qubits[n-i-1]), strategy = InsertStrategy.NEW)    
    
    for i in range(n-1,-1,-1):
        #Apply CR_k gates where j is the control and i is the target
        k=n-i
        for j in range(n-1,i,-1):
            #Define and apply CR_k gate  
            crk = CZPowGate(exponent = -2/2**(k))
            circuit.append(crk(qubits[j],qubits[i]),strategy = InsertStrategy.NEW)
            k-=1 #Increment k at each step
    
        #Apply Hadamard to the qubit
        circuit.append(H(qubits[i]), strategy = InsertStrategy.NEW)

In [6]:
def qft(n,qubits,circuit):
    #Define shorthand notation for Hadamard and Swap gates
    H = cirq.H
    swap = cirq.SWAP

    #For each qubit
    for i in range(n):
        #Apply Hadamard to the qubit
        circuit.append(H(qubits[i]))
        #Apply CR_j gates where j is the control and i is the target
        for j in range(i+1,n):
            #Define and apply CR_j gate
            crj = cirq.CZPowGate(exponent = 2/2**j)
            circuit.append(crj(qubits[j],qubits[i]))

    #Swap gates
    if n%2==0:
        k=int(n/2)
    else:
        k=int((n-1)/2)
        
    for i in range(k):
        circuit.append(swap(qubits[i],qubits[n-i-1]))

In [11]:
circuit = cirq.Circuit()

t=3 #Number of qubits in the a and b register
n=2*t-1 #Number of qubits in the output

#Create t control qubits
a = [cirq.LineQubit(i) for i in range(t)]
b = [cirq.LineQubit(i) for i in range(t, 2*t)]
#Create n target qubits
target = [cirq.LineQubit(i) for i in range(2*t,t+n) ]

qft(b ,control,circuit)
iqft(t,control,circuit)
circuit.append(X.on_each(target))
circuit.append(cirq.measure(*control, key='result'))
s=cirq.Simulator()
print('Sample the circuit:')
samples=s.run(circuit, repetitions=1000)
# Print a histogram of results
print(samples.histogram(key='result'))
print(circuit)

Sample the circuit:
Counter({0: 1000})
              ┌──────┐
0: ───H───@────@─────────────────────×───×────────────────────@─────────@────────H───M('result')───
          │    │                     │   │                    │         │            │
1: ───────@────┼────H────@───────────┼───┼───────@────────H───┼─────────@^-0.5───────M─────────────
               │         │           │   │       │            │                      │
2: ────────────@^0.5─────@^0.5───H───×───×───H───@^-0.5───────@^-0.25────────────────M─────────────

3: ───X────────────────────────────────────────────────────────────────────────────────────────────
              └──────┘
